In [72]:
# KBO 일정 크롤링
## 2024 UPDATE - 기존에 사용하던 BS 와 다르게 경기 일정은 JS 를 통해 동적으로 생성되기 때문에 Selenium 사용 필요해짐

### 1. 패키지 세팅 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup



In [6]:
## 임의의 날짜로 테스트

url = "https://m.sports.naver.com/kbaseball/schedule/index?date=2024-04-30"
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
contents = soup.find_all("li", {"class": "MatchBox_match_item__3_D0Q"})

contentArray = [];

for i in contents:
    item = i.find("div", {"class": "MatchBox_match_area__39dEr"})
    match = item.find_all("div", {"class": "MatchBoxTeamArea_name_info__2IaZV"})
    info = []
    
    for j in match:
        info.append(j.find("strong", {"class": "MatchBoxTeamArea_team__3aB4O"}).string)
    

    if len(info) < 1:
        break;

    contentArray.append({"home": info[1], "away": info[0]})

print(contentArray) #배열길이가 0이면 경기 없음

None


In [73]:
## 개막 날짜 넣어서 모든 경기 일정 불러오기
from datetime import date, timedelta

startDateString = '2024-03-24'
startDate = date.fromisoformat(startDateString)
endDateString = '2024-04-01' # 임의의 종료 날짜
endDate = date.fromisoformat(endDateString)

dates = [];
dateDiff = datetime(endDate.year, endDate.month, endDate.day) - datetime(startDate.year, startDate.month, startDate.day)

for i in range(0, dateDiff.days):
    dates.append((date.fromisoformat(startDateString) + timedelta(days = i)).isoformat())


In [ ]:
driver = webdriver.Chrome()

baseUrl = "https://m.sports.naver.com/kbaseball/schedule/index?date="

data = {}

for d in dates:
    url = ''.join([baseUrl, d])
    driver.get(url)
    try:
        e = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".MatchBoxTeamArea_team__3aB4O")))
        print(e.getText())
        soup = BeautifulSoup(driver.page_source, 'html.parser')
#         soup = BeautifulSoup(e,getText(), 'html.parser')
        contents = soup.find_all("li", {"class": "MatchBox_match_item__3_D0Q"})

        contentArray = [];

        for i in contents:
            item = i.find("div", {"class": "MatchBox_match_area__39dEr"})
            match = item.find_all("div", {"class": "MatchBoxTeamArea_name_info__2IaZV"})
            info = []

            for j in match:
                info.append(j.find("strong", {"class": "MatchBoxTeamArea_team__3aB4O"}).string)

            print(info)

    #         랜덤하게 경기가 안담기는 경우가 있음 -> 웹페이지 로딩되는 시간 때문?
    #         if len(info) < 1:
    #             continue;

            contentArray.append({"home": info[1], "away": info[0]})
#             continue

        data[d] = contentArray
        
    except:
        print("EXCEPTION!")

print(data) #배열길이가 0이면 경기 없음
driver.quit()


EXCEPTION!
